# 5章
(https://nlp100.github.io/ja/ch05.html)

In [4]:
from pathlib import Path

file_path = Path('Dataset/ai.ja.txt.parsed')
file_path.exists()

True

## 5.40, 41

In [2]:
class Morph(object):

    def __init__(
        self, surface: str, base: str, pos: str, pos1: str
    ) -> None:
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def __repr__(self) -> str:
        return f'{self.surface} [{self.base},{self.pos},{self.pos1}]'

In [3]:
class Chunk(object):

    def __init__(self, id_: int, morphemes: list, dst: int) -> None:
        self.cid = id_
        self.morphs = morphemes
        self.dst = dst
        self.srcs = []

    def __repr__(self) -> str:
        return '"{}" dst:{}, srcs:{}'.format(
            ' '.join((m.surface for m in self.morphs)), self.dst, self.srcs
        )


In [4]:
with file_path.open('r') as fp:
    sentences_raw = fp.read().split('EOS\n')

sentences_raw = [s for s in sentences_raw if s]

In [5]:
from collections import defaultdict
import re

pattern = re.compile(r'\n\*\s')
sentences_chunked = list(map(lambda x: pattern.split(x[2:]), sentences_raw))

sentences_morph = []
sentences_chunk = []

def parse_morph(line: str) -> Morph:
    surface, attr = line.split('\t')
    attr = attr.split(',')
    return Morph(surface, attr[6], attr[0], attr[1])

for sent_chunked in sentences_chunked[:2]:
    sent_morphs = []
    sent_chunks = []
    id2srcs = defaultdict(list)  # key: chunk id, value: srcsのリスト

    for chunk in sent_chunked:
        chunk = chunk.split('\n')
        morphs = [c for c in chunk[1:] if c]
        morphs = list(map(parse_morph, morphs))
        sent_morphs += morphs

        info = chunk[0].split()
        cid, dst = int(info[0]), int(info[1].rstrip('D'))
        id2srcs[dst].append(cid)
        sent_chunks.append(Chunk(cid, morphs, dst))

    for cid, srcs in id2srcs.items():
        sent_chunks[cid].srcs = srcs

    sentences_morph.append(sent_morphs)
    sentences_chunk.append(sent_chunks)


In [6]:
print(*sentences_morph[1], sep='\n')

人工 [人工,名詞,一般]
知能 [知能,名詞,一般]
（ [（,記号,括弧開]
じん [じん,名詞,一般]
こうち [こうち,名詞,一般]
のう [のう,助詞,終助詞]
、 [、,記号,読点]
、 [、,記号,読点]
AI [*,名詞,一般]
〈 [〈,記号,括弧開]
エーアイ [*,名詞,固有名詞]
〉 [〉,記号,括弧閉]
） [）,記号,括弧閉]
と [と,助詞,格助詞]
は [は,助詞,係助詞]
、 [、,記号,読点]
「 [「,記号,括弧開]
『 [『,記号,括弧開]
計算 [計算,名詞,サ変接続]
（ [（,記号,括弧開]
） [）,記号,括弧閉]
』 [』,記号,括弧閉]
という [という,助詞,格助詞]
概念 [概念,名詞,一般]
と [と,助詞,並立助詞]
『 [『,記号,括弧開]
コンピュータ [コンピュータ,名詞,一般]
（ [（,記号,括弧開]
） [）,記号,括弧閉]
』 [』,記号,括弧閉]
という [という,助詞,格助詞]
道具 [道具,名詞,一般]
を [を,助詞,格助詞]
用い [用いる,動詞,自立]
て [て,助詞,接続助詞]
『 [『,記号,括弧開]
知能 [知能,名詞,一般]
』 [』,記号,括弧閉]
を [を,助詞,格助詞]
研究 [研究,名詞,サ変接続]
する [する,動詞,自立]
計算 [計算,名詞,サ変接続]
機 [機,名詞,接尾]
科学 [科学,名詞,一般]
（ [（,記号,括弧開]
） [）,記号,括弧閉]
の [の,助詞,連体化]
一 [一,名詞,数]
分野 [分野,名詞,一般]
」 [」,記号,括弧閉]
を [を,助詞,格助詞]
指す [指す,動詞,自立]
語 [語,名詞,一般]
。 [。,記号,句点]
「 [「,記号,括弧開]
言語 [言語,名詞,一般]
の [の,助詞,連体化]
理解 [理解,名詞,サ変接続]
や [や,助詞,並立助詞]
推論 [推論,名詞,サ変接続]
、 [、,記号,読点]
問題 [問題,名詞,ナイ形容詞語幹]
解決 [解決,名詞,サ変接続]
など [など,助詞,副助詞]
の [の,助詞,連体化]
知的 [知的,名詞,一般]
行動 [行動,名詞,サ変接続]
を [を,助詞,格助詞]
人間 [人間,名詞,一般]
に [に,助詞,格助詞]
代わっ [代わる,動詞,自立]
て [て,助詞,接続

In [7]:
print(*sentences_chunk[1], sep='\n')

"人工 知能" dst:17, srcs:[]
"（ じん こうち のう 、 、" dst:17, srcs:[]
"AI" dst:3, srcs:[]
"〈 エーアイ 〉 ） と は 、" dst:17, srcs:[2]
"「 『 計算" dst:5, srcs:[]
"（ ） 』 という" dst:9, srcs:[4]
"概念 と" dst:9, srcs:[]
"『 コンピュータ" dst:8, srcs:[]
"（ ） 』 という" dst:9, srcs:[7]
"道具 を" dst:10, srcs:[5, 6, 8]
"用い て" dst:12, srcs:[9]
"『 知能 』 を" dst:12, srcs:[]
"研究 する" dst:13, srcs:[10, 11]
"計算 機 科学" dst:14, srcs:[12]
"（ ） の" dst:15, srcs:[13]
"一 分野 」 を" dst:16, srcs:[14]
"指す" dst:17, srcs:[15]
"語 。" dst:34, srcs:[0, 1, 3, 16]
"「 言語 の" dst:20, srcs:[]
"理解 や" dst:20, srcs:[]
"推論 、" dst:21, srcs:[18, 19]
"問題 解決 など の" dst:22, srcs:[20]
"知的 行動 を" dst:24, srcs:[21]
"人間 に" dst:24, srcs:[]
"代わっ て" dst:26, srcs:[22, 23]
"コンピューター に" dst:26, srcs:[]
"行わ せる" dst:27, srcs:[24, 25]
"技術 」 、 または 、" dst:34, srcs:[26]
"「 計算 機" dst:29, srcs:[]
"（ コンピュータ ） による" dst:31, srcs:[28]
"知的 な" dst:31, srcs:[]
"情報処理 システム の" dst:33, srcs:[29, 30]
"設計 や" dst:33, srcs:[]
"実現 に関する" dst:34, srcs:[31, 32]
"研究 分野 」 と も" dst:35, srcs:[17, 27, 33]
"さ れる 。" dst:-

In [8]:
sent_chunks = sentences_chunk[1]  # 2文目のみ処理
id2chunk = {chunk.cid: chunk for chunk in sent_chunks}

for chunk in sent_chunks:
    if chunk.dst == -1:
        continue
    src = ''.join((m.surface if m.pos != '記号' else '' for m in chunk.morphs))
    dst_chunk = id2chunk[chunk.dst]
    dst = ''.join((m.surface if m.pos != '記号' else '' for m in dst_chunk.morphs))
    print(f'{src}\t{dst}')

人工知能	語
じんこうちのう	語
AI	エーアイとは
エーアイとは	語
計算	という
という	道具を
概念と	道具を
コンピュータ	という
という	道具を
道具を	用いて
用いて	研究する
知能を	研究する
研究する	計算機科学
計算機科学	の
の	一分野を
一分野を	指す
指す	語
語	研究分野とも
言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
代わって	行わせる
コンピューターに	行わせる
行わせる	技術または
技術または	研究分野とも
計算機	コンピュータによる
コンピュータによる	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野とも
研究分野とも	される


In [9]:
for chunk in sent_chunks:
    if chunk.dst == -1:
        continue
    if '名詞' not in [m.pos for m in chunk.morphs]:
        continue
    src = ''.join((m.surface if m.pos != '記号' else '' for m in chunk.morphs))
    dst_chunk = id2chunk[chunk.dst]
    if '動詞' not in [m.pos for m in dst_chunk.morphs]:
        continue
    dst = ''.join((m.surface if m.pos != '記号' else '' for m in dst_chunk.morphs))
    print(f'{src}\t{dst}')

道具を	用いて
知能を	研究する
一分野を	指す
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
研究分野とも	される
